This notebook pulls wildfire point data from Cal Fire, then cleans the data.

In [2]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [267]:
response = urllib.request.urlopen('https://www.fire.ca.gov/incidents/2020/')
html = response.read()
soup = BeautifulSoup(html,'html.parser')
souplist = soup.text.split('create_map([')
sl2 = souplist[1].split(']);\n')
string = sl2[0].split('{')[1:]

In [268]:
list_of_incidents = []
for idx, inc in enumerate(string):
  if inc[:12] == '"IncidentId"':
    list_of_incidents.append(inc)

List of fire incidents

In [ ]:
#list_of_incidents

In [270]:
len(list_of_incidents)

260

Pull out relevant information

In [271]:
list_of_fires = []
for i in range(len(list_of_incidents)):
  single_incident = list_of_incidents[i][:-1].split(',')
  temp_list = []
  for j in single_incident:
    if 'Mercury' in j:
      print(single_incident)
    if '"Name":' in j:
      #print(f'{i},{j[8:-1]}')
      temp_list.append(j[8:-1])
    if 'StartedDate' in j:
      #print(j[15:-1])
      temp_list.append(j[15:-1])
    if 'Longitude' in j:
      #print(j[12:])
      temp_list.append(j[12:]) 
    if 'Latitude' in j:
      #print(j[11:])
      temp_list.append(j[11:])
    #print(temp_list)
  list_of_fires.append(temp_list)

In [272]:
#list_of_fires

In [273]:
len(list_of_fires)

260

In [274]:
import pandas as pd
df = pd.DataFrame(list_of_fires,columns=['Name', 'Lat', 'Lon', 'Date'])
df

,Name,Lat,Lon,Date
0,Bear Fire,40.511092,-123.6378411,2020-02-15
1,Antelope Fire,39.6923001,-120.3291855,2020-02-17
2,Beegum Fire,40.33583333,-122.84722222,2020-02-23
3,Wood Fire,40.62155,-120.49392,2020-02-23
4,Baseball Fire,39.75638899,-122.94111111,2020-02-25
...,...,...,...,...
255,Willow Fire,32.75277778,-116.9075,2020-12-02
256,Thomas Fire,41.591948,-120.374514,2020-12-03
257,Cornell Fire,34.33622,-119.078,2020-12-07
258,Sanderson Fire,33.93244,-117.03459,2020-12-13


Change dates to datetime. Change Lat/Lon to numeric.

In [275]:
df["Date"] = pd.to_datetime(df["Date"])
df["Lon"] = pd.to_numeric(df["Lon"])
df["Lat"] = pd.to_numeric(df["Lat"])

How many Lat/Long values are zero?

In [276]:
df[df['Lat']==0.0].reset_index()

,index,Name,Lat,Lon,Date


In [277]:
zeros = len(df[df['Lat']==0.0])
print(f'Found {zeros} records with no lat/long. New df should be size {len(df)-zeros}')

Found 0 records with no lat/long. New df should be size 260


Remove records with zero value Lat/Long

In [291]:
df_notna = df[df['Lat']!=0.0].sort_values(by='Name').reset_index()
df_notna

,index,Name,Lat,Lon,Date
0,163,166 Fire,34.934420,-119.579500,2020-08-03
1,250,257 Fire,41.443500,-120.870540,2020-11-17
2,194,3-19 Fire,39.276000,-123.013667,2020-08-18
3,181,3-2 Fire,35.519723,-120.801074,2020-08-16
4,187,4-8 Fire,39.555943,-122.501246,2020-08-17
...,...,...,...,...,...
255,255,Willow Fire,32.752778,-116.907500,2020-12-02
256,37,Willow Fire,37.994368,-122.002811,2020-06-06
257,3,Wood Fire,40.621550,-120.493920,2020-02-23
258,205,Woodward Fire,38.018089,-122.836701,2020-08-18


Inspect Lat/Long values

In [292]:
sorted(df_notna['Lat'].unique())

[-118.46115,
 32.580882,
 32.62212,
 32.73044,
 32.75277778,
 32.765615000000004,
 32.93432,
 33.27665,
 33.37579,
 33.40047,
 33.454896000000005,
 33.48429,
 33.551838000000004,
 33.565093,
 33.623968,
 33.638067,
 33.65263,
 33.73614,
 33.743842,
 33.764357000000004,
 33.773754,
 33.776033,
 33.782532,
 33.78275,
 33.82925,
 33.83167,
 33.861053999999996,
 33.877257,
 33.88831,
 33.893351,
 33.924085999999996,
 33.93244,
 33.93327,
 33.934137,
 33.969570000000004,
 33.99139,
 34.01389,
 34.014728000000005,
 34.028116,
 34.033429999999996,
 34.044470000000004,
 34.08056,
 34.135626,
 34.146195,
 34.160265,
 34.173629999999996,
 34.182435999999996,
 34.25143,
 34.29475,
 34.336220000000004,
 34.3428,
 34.39464,
 34.438759000000005,
 34.46182,
 34.467915000000005,
 34.47438,
 34.51853,
 34.542742,
 34.563,
 34.63309,
 34.65979,
 34.679,
 34.75844,
 34.773809,
 34.93442,
 34.93929,
 34.962524,
 35.143570000000004,
 35.17977,
 35.27678,
 35.30742,
 35.308259,
 35.310132,
 35.34237,
 35.35

In [293]:
sorted(df_notna['Lon'].unique())

[-124.19953999999998,
 -123.88658000000001,
 -123.6986222,
 -123.63784109999999,
 -123.48018,
 -123.437051,
 -123.3965,
 -123.262437,
 -123.21318000000001,
 -123.2111007,
 -123.12461,
 -123.021635,
 -123.013667,
 -122.97693000000001,
 -122.94111111,
 -122.84735900000001,
 -122.84722222,
 -122.84057,
 -122.836701,
 -122.792645,
 -122.69296000000001,
 -122.67299999999999,
 -122.65813,
 -122.637037,
 -122.56656000000001,
 -122.54981599999999,
 -122.536406,
 -122.50124609999999,
 -122.49745,
 -122.48525,
 -122.4393,
 -122.43905,
 -122.42735800000001,
 -122.40363400000001,
 -122.39033,
 -122.38453200000001,
 -122.37828,
 -122.35966299999998,
 -122.34200000000001,
 -122.25471,
 -122.22275,
 -122.205628,
 -122.18101999999999,
 -122.15631299999998,
 -122.14863999999999,
 -122.12447,
 -122.10431399999999,
 -122.09793300000001,
 -122.042199,
 -122.03820800000001,
 -122.00281100000001,
 -121.876277,
 -121.851895,
 -121.79348999999999,
 -121.726223,
 -121.68465,
 -121.68181000000001,
 -121.6415,
 

Check for out of bound values

In [294]:
df_notna[df_notna['Lat']<0]

,index,Name,Lat,Lon,Date
240,167,Texas Fire,-118.46115,-118.46115,2020-08-06


Switch Lat/Long values if they are obviously swapped

In [218]:
#df_notna.loc[222,'Lat']=34.015742
#df_notna.loc[222,'Lon']=-117.337331
#df_notna.loc[370,'Lat']=33.939428	
#df_notna.loc[370,'Lon']=-117.138741

In [302]:
#print(df_notna.loc[[92]])
#print(df_notna.loc[[61]])

In [295]:
df_notna[df_notna['Lon']>-114]

,index,Name,Lat,Lon,Date
73,21,Creek Fire,35.99940,-86.86160,2020-05-30
123,58,Hopland Fire,39.14783,-80.49163,2020-06-13


In [301]:
#df_notna.loc[56,'Lon']=-118.9082
#print(df_notna.loc[[56]])

Drop Lat/Longs that don't make sense

In [296]:
df_notna=df_notna.drop([240,73,123])

In [264]:
len(df_notna)

263

Recheck values

In [297]:
sorted(df_notna['Lat'].unique())

[32.580882,
 32.62212,
 32.73044,
 32.75277778,
 32.765615000000004,
 32.93432,
 33.27665,
 33.37579,
 33.40047,
 33.454896000000005,
 33.48429,
 33.551838000000004,
 33.565093,
 33.623968,
 33.638067,
 33.65263,
 33.73614,
 33.743842,
 33.764357000000004,
 33.773754,
 33.776033,
 33.782532,
 33.78275,
 33.82925,
 33.83167,
 33.861053999999996,
 33.877257,
 33.88831,
 33.893351,
 33.924085999999996,
 33.93244,
 33.93327,
 33.934137,
 33.969570000000004,
 33.99139,
 34.01389,
 34.014728000000005,
 34.028116,
 34.033429999999996,
 34.044470000000004,
 34.08056,
 34.135626,
 34.146195,
 34.160265,
 34.173629999999996,
 34.182435999999996,
 34.25143,
 34.29475,
 34.336220000000004,
 34.3428,
 34.39464,
 34.438759000000005,
 34.46182,
 34.467915000000005,
 34.47438,
 34.51853,
 34.542742,
 34.563,
 34.63309,
 34.65979,
 34.679,
 34.75844,
 34.773809,
 34.93442,
 34.93929,
 34.962524,
 35.143570000000004,
 35.17977,
 35.27678,
 35.30742,
 35.308259,
 35.310132,
 35.34237,
 35.351065000000006

In [298]:
sorted(df_notna['Lon'].unique())

[-124.19953999999998,
 -123.88658000000001,
 -123.6986222,
 -123.63784109999999,
 -123.48018,
 -123.437051,
 -123.3965,
 -123.262437,
 -123.21318000000001,
 -123.2111007,
 -123.12461,
 -123.021635,
 -123.013667,
 -122.97693000000001,
 -122.94111111,
 -122.84735900000001,
 -122.84722222,
 -122.84057,
 -122.836701,
 -122.792645,
 -122.69296000000001,
 -122.67299999999999,
 -122.65813,
 -122.637037,
 -122.56656000000001,
 -122.54981599999999,
 -122.536406,
 -122.50124609999999,
 -122.49745,
 -122.48525,
 -122.4393,
 -122.43905,
 -122.42735800000001,
 -122.40363400000001,
 -122.39033,
 -122.38453200000001,
 -122.37828,
 -122.35966299999998,
 -122.34200000000001,
 -122.25471,
 -122.22275,
 -122.205628,
 -122.18101999999999,
 -122.15631299999998,
 -122.14863999999999,
 -122.12447,
 -122.10431399999999,
 -122.09793300000001,
 -122.042199,
 -122.03820800000001,
 -122.00281100000001,
 -121.876277,
 -121.851895,
 -121.79348999999999,
 -121.726223,
 -121.68465,
 -121.68181000000001,
 -121.6415,
 

Drop index columns

In [299]:
df_notna.reset_index().drop(['level_0','index'],axis=1)

,Name,Lat,Lon,Date
0,166 Fire,34.934420,-119.579500,2020-08-03
1,257 Fire,41.443500,-120.870540,2020-11-17
2,3-19 Fire,39.276000,-123.013667,2020-08-18
3,3-2 Fire,35.519723,-120.801074,2020-08-16
4,4-8 Fire,39.555943,-122.501246,2020-08-17
...,...,...,...,...
252,Willow Fire,32.752778,-116.907500,2020-12-02
253,Willow Fire,37.994368,-122.002811,2020-06-06
254,Wood Fire,40.621550,-120.493920,2020-02-23
255,Woodward Fire,38.018089,-122.836701,2020-08-18


Save to csv

In [300]:
df_notna.to_csv('/content/cali_2020_fire_points.csv', encoding='utf-8')